In [62]:
# Import the dependencies
import pandas as pd
import numpy as np
import requests
from citipy import citipy 
from config import weather_api

In [63]:
#Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs
#Add the Latitude & Longitudes to a list.
coordinates = list(lat_lngs)

In [64]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
len(cities)

755

In [65]:
#Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api

In [66]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")

maragogi
Processing Record 5 of Set 7 | ulaangom
Processing Record 6 of Set 7 | asau
Processing Record 7 of Set 7 | terenos
Processing Record 8 of Set 7 | neiafu
Processing Record 9 of Set 7 | pamekasan
Processing Record 10 of Set 7 | bloemfontein
Processing Record 11 of Set 7 | nicolae balcescu
Processing Record 12 of Set 7 | petropavlovsk-kamchatskiy
Processing Record 13 of Set 7 | zhezkazgan
Processing Record 14 of Set 7 | copala
Processing Record 15 of Set 7 | san policarpo
Processing Record 16 of Set 7 | villamontes
Processing Record 17 of Set 7 | trat
Processing Record 18 of Set 7 | ishigaki
Processing Record 19 of Set 7 | kloulklubed
Processing Record 20 of Set 7 | quepos
Processing Record 21 of Set 7 | kofarnihon
City not found. Skipping...
Processing Record 22 of Set 7 | grand river south east
City not found. Skipping...
Processing Record 23 of Set 7 | salalah
Processing Record 24 of Set 7 | constitucion
Processing Record 25 of Set 7 | pangnirtung
Processing Record 26 of Set 7

In [67]:
len(city_data)

682

In [68]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Swan River,CA,52.1058,-101.2676,62.64,60,40,3.74,scattered clouds
1,Ushuaia,AR,-54.8000,-68.3000,37.06,65,0,5.75,clear sky
2,Busselton,AU,-33.6500,115.3333,49.08,79,100,8.30,overcast clouds
3,Kamenka,RU,51.3223,42.7678,66.42,75,0,8.32,clear sky
4,Praia Da Vitoria,PT,38.7333,-27.0667,68.25,83,40,6.38,scattered clouds
5,Jiayuguan,CN,39.8167,98.3000,73.45,29,79,8.66,broken clouds
6,Bluff,NZ,-46.6000,168.3333,48.79,62,100,15.73,overcast clouds
7,Tasiilaq,GL,65.6145,-37.6368,41.16,81,3,3.89,clear sky
8,Arraial Do Cabo,BR,-22.9661,-42.0278,69.40,79,11,15.55,few clouds
9,Belyy Yar,RU,53.6039,91.3903,58.89,72,0,4.47,clear sky


In [69]:
# Create the output file (CSV).
output_data_file = "weatherpy_database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")